In [1]:
import os
import numpy as np
import cv2 
from PIL import Image
import scipy.io as sio
from sklearn import preprocessing

In [6]:
for c in classes:
    i = 0
    for r, d, f in os.walk('./data/matdata'):
        for file in f:
            t = file.split('_')[0]
            if t == c:
                i = i + 1
                os.rename(os.path.join(r, file), './data/matdata/' + c + "_{:03}".format(i) + '.mat') 

In [2]:
def load_value_file(file_path):
    with open(file_path, 'r') as input_file:
        value = float(input_file.read().rstrip('\n\r'))

    return value

def load(files):
    imgs = []
    for file in files:
        img = cv2.imread(file)
        imgs.append(img)
    return np.asarray(imgs)
        
def save(imgs, targe, index, label):
    imgs = imgs.reshape(imgs.shape[0], 28*28*3)
    min_max_scaler = preprocessing.MinMaxScaler(feature_range=(-1, 1))
    imgs = min_max_scaler.fit_transform(imgs.T).transpose()
    labels = []
    for i in range(imgs.shape[0]):
        labels.append(label)
    
    labels = np.asarray(labels)
    labels = labels.reshape(labels.shape[0], 1)
    
    data = np.concatenate((labels, imgs), axis = 1)
    info = {}
    info['data'] = data
    savefile = './data/matdata/' + targe + '_{0:05}.mat'
    sio.savemat(savefile.format(index), info)
    
def transform(img, newsize):
    image_width = img.size[0]
    image_height = img.size[1]
    min_length = min(image_width, image_height)
    crop_size = int(min_length * 0.6)
        
    center_x = image_width // 2
    center_y = image_height // 2
    box_half = crop_size // 2
    x1 = center_x - box_half
    y1 = center_y - box_half
    x2 = center_x + box_half
    y2 = center_y + box_half
    img = img.crop((x1, y1, x2, y2))
    return img.resize((newsize, newsize), Image.BILINEAR)
        
def transforms(imgs, newsize):
    for img in imgs:
        transform(img, newsize)

In [3]:
path = './data/data/ucf101/jpg/'

folders = []
for r, d, f in os.walk(path):
    for folder in d:
        if 'v_' in folder:
            folders.append(os.path.join(r, folder))

In [4]:
classes = []
for folder in folders:
    targe = folder.split('/')[5]
    if not targe in classes:
        classes.append(targe)

In [5]:
index = 0
for folder in folders:
    fs = []
    for r, d, f in os.walk(folder):
        for file in f:
            if 'n_frames' in file:
                n_frames = int(load_value_file(os.path.join(r, file)))
    for i in range(1, n_frames + 1):
        file = os.path.join(folder, 'image_0{0:04}.jpg'.format(i))
        fs.append(file)
    targe = folder.split('/')[5]
    label = classes.index(targe)
    data = load(fs)
    save(data, targe, index, label)
    index = index + 1

/usr/lib/python3/dist-packages/sklearn/utils/validation.py:475: DataConversionWarning: Data with input dtype uint8 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)
